# Personal Finance Subreddit Capstone Project

## Data Wrangling

The goal of this notebook is to extract data containing information about the personal finance subreddit from Reddit's API, select interesting features that can be relevant to the project and clean the dataset so that it is ready for data exploration.

- **Import the necessary packages**

In [1]:
import praw
import pandas as pd
import numpy as np
import datetime
import json
import requests
import string
import time
import datetime
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline

- **Extract the data from reddit using pushshift**

In [ ]:
# Variables
sub    = 'personalfinance'     # name of the subreddit you would like to scrape
after  = '2018-08-01'    # earliest date that will be scraped
before = '2018-09-19'    # latest date that will be scraped
fast   = True           # True will be faster, won't pull upvote ratio

In [ ]:
# Initiate sqlite
sql = sqlite3.connect('personalfinance_.db')
cur = sql.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS posts (name TEXT, title TEXT, readable_utc TEXT, permalink TEXT, domain TEXT, url TEXT, author TEXT, score TEXT, upvote_ratio TEXT, num_comments TEXT)')
sql.commit()
print('Loaded SQL Database and Tables')

# Convert the specified dates to strptime
after = time.mktime(datetime.datetime.strptime(after, '%Y-%m-%d').timetuple())
after = int(after)
readable_after = time.strftime('%d %b %Y %I:%M %p', time.localtime(after))
before = time.mktime(datetime.datetime.strptime(before, '%Y-%m-%d').timetuple())
before = int(before) + 86399
readable_before = time.strftime('%d %b %Y %I:%M %p', time.localtime(before))
print('Searching for posts between ' + readable_after + ' and ' + readable_before + '.')
currentDate = before

Using pushshift will allow us to retrieve valuable information from reddit submissions including:
- Submission ID
- Submission title
- Submission date
- Submission permalink
- Submission domain
- Submission url
- Submission author
- Submission score (upvotes)
- Submission upvote_ratio (ratio of upvotes to downvotes)
- Submission number of comments

However, it does not provide us with the flair information.

In [ ]:
# Perform a new full pull from Pushshift
def newpull(thisBefore):
    global currentDate
    readable_thisBefore = time.strftime('%d %b %Y %I:%M %p', time.localtime(thisBefore))
    print('Searching posts before ' + str(readable_thisBefore))
    url = 'http://api.pushshift.io/reddit/search/submission/?subreddit=' + sub + '&size=500&before=' + str(thisBefore)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print('    Discussion: HTML Error - ', response.status_code)
        time.sleep(60)
        return
    curJSON = response.json()

    # Update each Pushshift result with Reddit data
    for child in curJSON['data']:

        # Check to see if already added
        name = str(child['id'])
        cur.execute('SELECT * FROM posts WHERE name == ?', [name])
        if cur.fetchone():
            print(str(child['id']) + ' skipped (already in database)')
            continue

        # If not, get more data
        if fast is True:
            searchURL = 'http://reddit.com/by_id/t3_'
        else:
            searchURL = 'http://reddit.com/'
        url = searchURL + str(name) + '.json'
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'}
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print('    Discussion: HTML Error - ', response.status_code)
            time.sleep(60)
            break
        postJSON = response.json()
        if fast is True:
            jsonStart = postJSON
        else:
            jsonStart = postJSON[0]

        # Check to see if Date has passed
        global currentDate
        created_utc = jsonStart['data']['children'][0]['data']['created_utc']
        currentDate = int(created_utc)
        if currentDate <= after:
            break

        # If not, process remaining data
        try:
            title = str(jsonStart['data']['children'][0]['data']['title'])  # Checks for emojis and other non-printable characters
        except UnicodeEncodeError:
            title = ''.join(c for c in str(jsonStart['data']['children'][0]['data']['title']) if c in string.printable)
        readable_utc = time.strftime('%d %b %Y %I:%M %p', time.localtime(created_utc))
        permalink    = (str(jsonStart['data']['children'][0]['data']['permalink']))
        domain       = (str(jsonStart['data']['children'][0]['data']['domain']))
        url          = (str(jsonStart['data']['children'][0]['data']['url']))
        author       = (str(jsonStart['data']['children'][0]['data']['author']))
        score        = (str(jsonStart['data']['children'][0]['data']['score']))
        num_comments = (str(jsonStart['data']['children'][0]['data']['num_comments']))
        if fast is True:
            upvote_ratio = 0
        else:
            upvote_ratio = (str(jsonStart['data']['children'][0]['data']['upvote_ratio']))

        # Write it to SQL Database
        cur.execute('INSERT INTO posts VALUES(?,?,?,?,?,?,?,?,?,?)', [name, title, readable_utc, permalink, domain, url, author, score, upvote_ratio, num_comments])
        sql.commit()

# Run the newpull
while currentDate >= after:
    newpull(currentDate)

Now that we have pulled the data from pushshift, we will need to create a dataframe which will store the relevant information (title, date, time, upvotes, id).

In [ ]:
# Reconnect to sqlite
connection = sqlite3.connect("personalfinance_.db") 
  
# Cursor object 
crsr = connection.cursor() 
  
# Execute the command to fetch all the data from the table posts 
crsr.execute("SELECT * FROM posts")  
  
# Store all the fetched data in the ans variable 
ans= crsr.fetchall()  

# Create empty dataframe
columns = ['title']
index = range(0,2)
df = pd.DataFrame(index = index, columns = columns)
df = df.fillna(0)

# Create new columns and extract the relevant data 
for n, i in enumerate(ans):
    # Create title column
    df.loc[n , 'title'] = i[1]
    # Create date column
    df.loc[n , 'date'] = i[2][:-8]
    # Create time column
    df.loc[n , 'time'] = i[2][-8:]
    # Create upvote column
    df.loc[n , 'upvotes'] = i[7]
    # Create id column
    df.loc[n , 'id'] = i[0]

- **Retrieve flair information from Reddit's API**

As mentioned before, we still need to extract the flair (which indicates the topic of each submission) from each post. To do this, we will have to initiate a Reddit instance using praw (which gives access to Reddit's API).

In [ ]:
# Create new Reddit instance
reddit = praw.Reddit(client_id='',
                     client_secret='',
                     user_agent='',
                    username = '',
                    password = '')

Create a for-loop that will check and return the appropriate flair for each submission by using its ID as verification.

In [ ]:
for a,b in enumerate(df.id):
    df.loc[a, 'topic'] = reddit.submission(id = "{}".format(b)).link_flair_text
    try:
        print(a,',', df.loc[a, 'topic'])
    except:
        print('None')

- **Convert Dates into datetime format**

Since the data is given as a string, we will need to convert it into datatime format.

In [ ]:
from datetime import datetime
for a,b in enumerate(df.date):
     df.loc[a, 'date'] = datetime.strptime(b, '%d %b %Y ').date()

In [ ]:
df = df.sort_values(by=['date', 'time'], ascending = False).reset_index().drop('index',axis='columns')

- **Examine basic information**

Let's begin by taking a peek at the dataframe's contents.

In [3]:
print('Dimensions of the dataframe: {}'.format(df.shape))
print(20*'-')
df.info()

Dimensions of the dataframe: (10183, 6)
--------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10183 entries, 0 to 10182
Data columns (total 6 columns):
title      10183 non-null object
date       10183 non-null object
time       10183 non-null object
upvotes    10183 non-null int64
id         10183 non-null object
topic      9532 non-null object
dtypes: int64(1), object(5)
memory usage: 556.9+ KB


The only column with missing data is 'topic' due to the fact that some posts have been removed and therefore their flairs no longer show up during extraction. To deal with this issue, we can simply fill in 'unknown'.

In [ ]:
df['topic'].fillna('unknown', inplace = True)

Finally, let's save the data as a .csv file for later use.

In [ ]:
# Save as .csv file
df.to_csv(r'C:\Users\joshu\Downloads\Data\reddit\reddit_pf1.csv')